In [1]:
#importing libraries
from selenium import webdriver
import sys
sys.path.insert(0,'/usr/bin/chromedriver')
import time
from selenium.common.exceptions import NoSuchElementException , ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml
import re
import pandas as pd
import json
import time

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
def CheckUnique(links):

    
        count_dict = {link:links.count(link) for link in links if links.count(link) > 1}
        clean_links = [link for link in links if link not in list(count_dict.keys())]
        return clean_links

def ScrapeLinks(driver , start_pg , end_pg ,total_pages):

        #Total pages are the total web pages on the website not the total you want to scrape
        if end_pg > total_pages:
            print("Enter valid Page Indexes")
        br = driver
        scr_links = []
        url_head = 'https://www.glassdoor.co.in'
        #br = self.Initialize_Driver()
        for pg_num in range(start_pg, end_pg+1):
            url = 'https://www.glassdoor.co.in/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC1132348_KO9,23_IP'+str(i)+'.htm?fromAge=14'
            br.get(url)
            br.implicitly_wait(5)

            soup = BeautifulSoup(br.page_source , 'lxml')

            #print(f"Scraping Page Number : {pg_num}")
            for a_tag in soup.find_all('a' , href = True):
                pattern = re.compile('/partner/jobListing')
                if pattern.match(str(a_tag['href'])):
                    scr_links.append(url_head + str(a_tag['href']))
        
        op_links = CheckUnique(scr_links)
        if len(op_links) == ((end_pg-start_pg) +1)*30 :
            print("Scraping of links is completed")
            return op_links
        

        

def ScrapeData(num_jobs,driver , links):

        br = driver
        #data_df = df
        #cleaned_links = ScrapeLinks(br)
        #if data_df is None:
        data_df = pd.DataFrame(columns = ['Job_title' , 'Company' , 'State' , 'City' , 'Min_Salary' ,'Max_Salary' ,'Job_Desc' , 
                                         'Industry'  , 'Rating' , 'Date_Posted' , 'Valid_until' , 'Job_Type' ,
                                          ])
        
        start = time.time()
        print("Gathering Information")
        for job in range(num_jobs):

            #print(f'Extracting information about job : {job}')
            br.get(links[job])
            br.implicitly_wait(10)

            soup = BeautifulSoup(br.page_source , 'lxml')

            try:
                
                #The rest of the data is available in scrip = 'application/ld+json' i.e in  json file
                json_file = soup.find('script' , {'type':'application/ld+json'})
                op_json = json.loads(str(json_file.text) , strict = False)
                
                #title
                try:
                    title = op_json['title']
                    #print(title)
                except:
                    title  = None
                
                #Dateposted
                try:
                    post_date = op_json['datePosted']
                except:
                    post_date = None
                
                #type
                try:
                    job_type = op_json['employmentType']
                except:
                    job_type = None

                #Annual Salary Range
                try:
                    min_salary = int(op_json['estimatedSalary']['value']['minValue'])
                    max_salary = int(op_json['estimatedSalary']['value']['maxValue'])
                except:
                    min_salary = -1
                    max_salary = -1
                
                #Job Posting validity date(Y-M-D)
                try:
                    valid_date = op_json['validThrough']
                except:
                    valid_date = None
                
                #Industry
                try:
                    industry = op_json['industry']
                except:
                    industry = None
                
                #Location
                try:
                    city = op_json['jobLocation']['address']['addressLocality']
                    state = op_json['jobLocation']['address']['addressRegion']
                except:
                    city , state = None , None

                #Company
                try:
                    company = op_json['hiringOrganization']['name']
                except:
                    company = None
                
                #Let's get Description
                try:
                    desc = soup.find(class_ = 'desc').text
                except:
                    desc = None
                try:
                    rating = soup.find('span' , {'class' : 'css-1pmc6te e11nt52q3'}).text.replace('★' , '')
                
                except:
                    rating = None


            
                data_df = data_df.append({'Job_title' : title,
                                      'Company' : company,
                                      'State' : state,
                                      'City' : city,
                                      'Min_Salary':min_salary,
                                      'Max_Salary':max_salary,
                                      'Job_Desc' : desc,
                                      'Industry':industry,
                                      'Rating':rating,
                                      'Date_Posted' : post_date,
                                      'Valid_until' : valid_date,
                                      'Job_Type' :job_type} , ignore_index = True)
            except:
                pass
        
        #driver.close()
        print(f"Scraping Completed for {data_df.shape[0]} jobs")
        print(f"Time Required : {time.time() - start} seconds")
        
        return data_df

In [3]:
#Please Specify number of jobs as an excessive number of requests made by the computer Ip can potentially block you
#glassdoor has 30 listing per page, so you can calculate manually how much is the total amount of jobs present i.e (num_pages*30)
num_jobs = 150
start_pg =1 
end_pg = 30
total_pages = 30

service = Service(executable_path='/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome(service=service, options=options)

links = ScrapeLinks(driver,start_pg,end_pg , 30)

NameError: name 'i' is not defined

In [ ]:
links[:5]

In [ ]:
total_jobs = 30*(end_pg - start_pg+1)
f_df = pd.DataFrame(columns = ['Job_title' , 'Company' , 'State' , 'City' , 'Min_Salary' ,'Max_Salary' ,'Job_Desc' , 
                                         'Industry'  , 'Rating' , 'Date_Posted' , 'Valid_until' , 'Job_Type' ,
                                          ])
dummy_df  = ScrapeData(10 , driver , links)
#Exctract 150 jobs every 1 mins 
#for i in range(total_jobs//num_jobs):
     #df = ScrapeData(num_jobs , driver , links[num_jobs*i : num_jobs*(i+1)])
     #f_df = f_df.append(df)
     #time.sleep(30)

In [ ]:
#Save the file to csv
f_df.to_csv('Data_Job_SG' , index = False)